Here is a demo for how to calibrate the Dual-Rotating Retarder Polarimeter. This is a quick method to correct for imperfections in the optical components of the instrument which can be done without removing any optics. This method is suitable to reduce the error in an experimentally determined Mueller matrix by around one order of magnitude. To demonstrate this, we will need to use a few Python packages.

In [104]:
import numpy as np
from katsu.mueller import *
from katsu.polarimetry import *
import random

The basic idea behind this method is to create a realistic Mueller matrix model for the DRRP without having to individually measure the Mueller matrix of each component. Data reduction with the DRRP has three key components: a Mueller matrix representing the system, a matrix for the sample, and intensity measurements taken of the sample. This can be written roughly as:
\begin{equation}
    M_{sample} = M_{system}I_{measured}
\end{equation}
For calibration, we want to find $M_{system}$ from known $I_{measured}$ and $M_{sample}$. The easiest known sample to use is air, which corresponds with the identity matrix: 

In [105]:
# Mueller matrix of the sample (air)
I = np.identity(4)
print(I)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


We take measuremetns of air as usual with the DRRP to get $I_{measured}$. Now the only unknown in Equation (1) is the system matrix. Let's find it in an example by simulating data taken with non-ideal components. Here, "non-ideal" refers to deviations in the axis angle of the linear polarizers and waveplates, and the retardance of the waveplates. With perfect alignment, these angles should be zero and retardance should be pi/2. Let's add some small deviations to these values. 

In [88]:
# Define small deviations (could be anything)
err1 = 2*random.random()-1
err2 = 2*random.random()-1
err3 = 2*random.random()-1
err4 = 2*random.random()-1
err5 = 2*random.random()-1
error_terms = np.array([err1, err2, err3, err4, err5])

In [13]:
# Define angles to rotate the quarter-wave plate, taking 46 measurements
theta = np.linspace(0, np.pi, 46)

We want to know what the measured intensity would look like for a system with these error terms. Thankfully, we can do this easily using simulation functions from katsu! The horizontal intensities indicate light detected from the horizontal polarization of the Wollaston prism, and likewise for vertical. We will need both for the data reduction process.

In [91]:
I_hor_sim = np.array(single_output_simulation_function(theta, err1, 0, err2, err3, err4, err5, LPA_angle=0, M_in=None))
I_vert_sim = np.array(single_output_simulation_function(theta, err1, 0, err2, err3, err4, err5, LPA_angle=np.pi/2, M_in=None))

Ok, let's try using this data to recreate our sample matrix. Since we measured air, we should get the identity matrix as our output. In this case, let's not do any correction and set the five inputs for error terms to zero. These are the values that will be used in the system's matrix model, which means perfect components if the terms are zero. 

In [102]:
np.set_printoptions(precision=5, suppress=True) # suppress scientific notation
uncalibrated = q_calibrated_full_mueller_polarimetry(theta, 0, 0, 0, 0, 0, I_vert_sim, I_hor_sim, M_in=None) # No correction for error terms
uncalibrated = uncalibrated/np.max(np.abs(uncalibrated))
print(uncalibrated)
print("RMS error in the uncalibrated matrix: ", RMS_calculator(uncalibrated))

[[ 1.      -0.       0.      -0.     ]
 [ 0.10977  0.04792  0.11093 -0.     ]
 [ 0.85257 -0.19214  0.93569 -0.     ]
 [-0.      -0.       0.       0.14377]]
RMS error in the uncalibrated matrix:  0.38986135695283236


Now let's recreate the sample matrix using the actual error terms in our model. 

In [103]:
ideal = q_calibrated_full_mueller_polarimetry(theta, err1, err2, err3, err4, err5, I_vert_sim, I_hor_sim, M_in=None)
ideal  = ideal/np.max(np.abs(ideal))
print(ideal)
print("RMS error in the calibrated matrix: ", RMS_calculator(ideal))

[[ 1. -0. -0.  0.]
 [-0.  1. -0.  0.]
 [-0. -0.  1.  0.]
 [-0. -0.  0.  1.]]
RMS error in the calibrated matrix:  8.899372362315711e-16


As you can see, with the right error terms the correction is almost perfect! Now, if we don't know the error terms beforehand, how close can we get to finding them? We will use the measured intensities and a best fit of our matrix model to do this. 

In [117]:
ICal = I_hor_sim + I_vert_sim
QCal = I_hor_sim - I_vert_sim
initial_guess = [0, 0, 0, 0, 0]
parameter_bounds = ([-np.pi, -np.pi, -np.pi, -np.pi/2, -np.pi/2], [np.pi, np.pi, np.pi, np.pi/2, np.pi/2])

# Find parameters from calibration 
normalized_QCal = QCal/1 # With this simulated data, the input intensity is normalized to 1
popt, pcov = curve_fit(q_output_simulation_function, theta, normalized_QCal, p0=initial_guess, bounds=parameter_bounds)
print(popt, "Fit parameters for a1, w1, w2, r1, and r2. 1 for generator, 2 for analyzer")

[-0.93221 -1.20052 -0.68379 -0.35625  0.48562] Fit parameters for a1, w1, w2, r1, and r2. 1 for generator, 2 for analyzer


We have an answer! For comparison, the actual error terms were:

In [118]:
print(error_terms)

[-0.93221  0.37027  0.88701 -0.35625  0.48562]


Clearly this method isn't perfect because there are degeneracies between the error terms that should be explored. However, it does approach the right answer. Using these terms, let's see what the corrected sample matrix would look like.

In [114]:
calibrated = q_calibrated_full_mueller_polarimetry(theta, popt[0], popt[1], popt[2], popt[3], popt[4], I_vert_sim, I_hor_sim, M_in=None)
calibrated  = calibrated/np.max(np.abs(calibrated))
print(calibrated)
print("RMS error in the calibrated matrix: ", RMS_calculator(calibrated))

[[ 1.  0. -0.  0.]
 [-0.  1. -0. -0.]
 [-0. -0.  1. -0.]
 [ 0.  0.  0.  1.]]
RMS error in the calibrated matrix:  3.817153490539245e-12


Even without the exact error parameters, the correction is very good!

In [115]:
ICal = I_hor_sim + I_vert_sim
QCal = I_hor_sim - I_vert_sim
initial_guess = [0, 0, 0, 0, 0]
parameter_bounds = ([-np.pi, -np.pi, -np.pi, -np.pi/2, -np.pi/2], [np.pi, np.pi, np.pi, np.pi/2, np.pi/2])

# Find parameters from calibration 
normalized_QCal = QCal/(max(ICal)) # This line is different
popt, pcov = curve_fit(q_output_simulation_function, theta, normalized_QCal, p0=initial_guess, bounds=parameter_bounds)
print(popt, "Fit parameters for a1, w1, w2, r1, and r2. 1 for generator, 2 for analyzer")

[-0.97481 -1.23917 -0.69925 -0.28576  0.45679] Fit parameters for a1, w1, w2, r1, and r2. 1 for generator, 2 for analyzer


In [116]:
calibrated = q_calibrated_full_mueller_polarimetry(theta, popt[0], popt[1], popt[2], popt[3], popt[4], I_vert_sim, I_hor_sim, M_in=None)
calibrated  = calibrated/np.max(np.abs(calibrated))
print(calibrated)
print("RMS error in the calibrated matrix: ", RMS_calculator(calibrated))

[[ 1.      -0.      -0.      -0.     ]
 [-0.02223  0.91218 -0.01879  0.00057]
 [-0.09186  0.00933  0.92334  0.00046]
 [-0.00077 -0.00561 -0.00076  0.96102]]
RMS error in the calibrated matrix:  0.039141745038295615


The RMS error with this approximation is still an order of magnitude better than the uncalibrated case.

**Final Note**

In reality, the optical components will likely have imperfections in addition to the ones mentioned here. I believe the five error terms I identified are the dominant sources of error, but it's worth noting that other errors would not be corrected with this method. We could expand on this by adding more error terms but it's unclear how more terms would impact the effectiveness of the fit.